In [4]:
import pandas as pd
import os 
from loguru import logger 



In [3]:
CSV_FILE_PATH = "/quotes_db.csv"

In [ ]:
def write_db(df: pd.DataFrame):
    df.to_csv(CSV_FILE_PATH, index=True, index_label='id')

def read_db()->pd.DataFrame:
    df = pd.read_csv(CSV_FILE_PATH, index_cols='id')
    return df

def initialize_db():
    df = pd.DataFrame(columns=['id', 'text'])
    df = df.set_index('id')
    write_db(df)


initialize_db()

PermissionError: [Errno 13] Permission denied: '/quotes_db.csv'